# Spark ML (Jahr-Join): Verkehr ↔ Schadstoffe (Wien) – angepasst auf deine CSVs

Dieses Notebook ist für **deine beiden CSV-Exports** gebaut:

**Verkehr.csv Header**
`_id;NUTS1;NUTS2;NUTS3;DISTRICT_CODE;SUB_DISTRICT_CODE;YEAR;UNIT;REF_YEAR;ROAD_TRAFFIC;SCWR_CALC;_imported_at`

**Schadstoff.csv Header**
`_id;Region;Schadstoff;Einheit;NFR_Code;Trendbericht_Sektor;Quelle;Datenstand;Jahr; Werte ;_imported_at`

Besonderheiten:
- Separator: `;`
- Zahlenformat kann `.` als Tausender und `,` als Dezimal haben → wird zu `double` gecastet
- Kein Timestamp → Join über **YEAR ↔ Jahr**
- Schadstoffe: nur **Region == Wien**
- Verkehr: ist ohnehin Wien → wir setzen Region-Spalte auf `"Wien"`
- Fehlende Jahre im Verkehr: Join ist standardmäßig `left` (Schadstoff-Jahre bleiben)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

## 1) Spark Session

In [ ]:
spark = (SparkSession.builder
         .appName("air-traffic-year-join-ml")
         .getOrCreate())

spark.version

## 2) Pfade setzen (CSV)

In [ ]:
# ---------------------------
# HIER ANPASSEN
# ---------------------------
TRAFFIC_CSV_PATH = "./export/Verkehr.csv"
AIR_CSV_PATH     = "./export/Schadstoff.csv"

CSV_SEP = ";"
HAS_HEADER = True

## 3) CSV laden (alles als String, dann sauber casten)

In [ ]:
traffic_raw = (spark.read
               .option("header", str(HAS_HEADER).lower())
               .option("sep", CSV_SEP)
               .option("inferSchema", "false")
               .csv(TRAFFIC_CSV_PATH))

air_raw = (spark.read
           .option("header", str(HAS_HEADER).lower())
           .option("sep", CSV_SEP)
           .option("inferSchema", "false")
           .csv(AIR_CSV_PATH))

print("traffic rows:", traffic_raw.count(), "cols:", len(traffic_raw.columns))
print("air rows:", air_raw.count(), "cols:", len(air_raw.columns))

traffic_raw.show(5, truncate=False)
air_raw.show(5, truncate=False)

## 4) Spaltennamen trimmen (wichtig wegen ` Werte `)

In [ ]:
traffic_raw = traffic_raw.toDF(*[c.strip() for c in traffic_raw.columns])
air_raw = air_raw.toDF(*[c.strip() for c in air_raw.columns])

print("Traffic columns:", traffic_raw.columns)
print("Air columns:", air_raw.columns)

## 5) Helper: deutsches Zahlenformat → double

In [ ]:
def cast_de_number(df, colname: str, to_type="double"):
    # '.' als Tausender raus, ',' -> '.'
    cleaned = F.regexp_replace(F.col(colname).cast("string"), r"\.", "")
    cleaned = F.regexp_replace(cleaned, r",", ".")
    return df.withColumn(colname, cleaned.cast(to_type))

## 6) Fixe Spalten für deine Dateien

In [ ]:
# ---------------------------
# Fix: Spalten aus deinen CSVs
# ---------------------------

# Verkehr.csv
TRAFFIC_YEAR_COL  = "YEAR"
TRAFFIC_VALUE_COL = "ROAD_TRAFFIC"

# Schadstoff.csv
AIR_REGION_COL    = "Region"
AIR_YEAR_COL      = "Jahr"
AIR_POLLUTANT_COL = "Schadstoff"
AIR_VALUE_COL     = "Werte"   # nach trimmen heißt es "Werte"

TARGET_REGION = "Wien"

## 7) Verkehr vorbereiten (YEAR int, ROAD_TRAFFIC double, Region='Wien')

In [ ]:
traffic = traffic_raw

traffic = traffic.withColumn(TRAFFIC_YEAR_COL, F.col(TRAFFIC_YEAR_COL).cast("int"))
traffic = cast_de_number(traffic, TRAFFIC_VALUE_COL, "double")

# Verkehr ist laut dir ohnehin Wien -> Region-Spalte setzen
traffic = traffic.withColumn("Region", F.lit(TARGET_REGION))

traffic.select(TRAFFIC_YEAR_COL, "Region", TRAFFIC_VALUE_COL).show(10, truncate=False)

## 8) Schadstoffe vorbereiten (Region=Wien, Jahr int, Werte double)

In [ ]:
air = air_raw

air = air.withColumn(AIR_YEAR_COL, F.col(AIR_YEAR_COL).cast("int"))
air = air.filter(F.trim(F.col(AIR_REGION_COL)) == TARGET_REGION)

air = cast_de_number(air, AIR_VALUE_COL, "double")

air.select(AIR_REGION_COL, AIR_YEAR_COL, AIR_POLLUTANT_COL, AIR_VALUE_COL).show(10, truncate=False)

## 9) Aggregation pro Jahr

In [ ]:
# Verkehr pro Jahr (falls mehrere Zeilen pro Jahr vorhanden sind: avg)
traffic_year = (traffic
    .groupBy(TRAFFIC_YEAR_COL, "Region")
    .agg(F.avg(F.col(TRAFFIC_VALUE_COL)).alias("traffic_road_traffic_avg"))
)

traffic_year.orderBy(TRAFFIC_YEAR_COL).show(30, truncate=False)

In [ ]:
# Schadstoffe pro Jahr & Schadstoff (avg)
air_year = (air
    .groupBy(AIR_YEAR_COL, AIR_POLLUTANT_COL)
    .agg(F.avg(F.col(AIR_VALUE_COL)).alias("poll_value_avg"))
)

air_year.orderBy(AIR_YEAR_COL, AIR_POLLUTANT_COL).show(30, truncate=False)

## 10) Pivot: pro Jahr eine Zeile, pro Schadstoff eine Spalte

In [ ]:
air_year_pivot = (air_year
    .groupBy(AIR_YEAR_COL)
    .pivot(AIR_POLLUTANT_COL)
    .agg(F.first("poll_value_avg"))
)

air_year_pivot.orderBy(AIR_YEAR_COL).show(30, truncate=False)
print("Pivot columns:", air_year_pivot.columns)

## 11) Join über Jahr (left: alle Schadstoff-Jahre behalten)

In [ ]:
joined = (air_year_pivot
    .join(
        traffic_year.withColumnRenamed(TRAFFIC_YEAR_COL, "YEAR_join"),
        air_year_pivot[AIR_YEAR_COL] == F.col("YEAR_join"),
        how="left"
    )
    .drop("YEAR_join")
)

joined.orderBy(AIR_YEAR_COL).show(50, truncate=False)

## 12) Quick-Checks: welche Jahre fehlen im Verkehr?

In [ ]:
# Jahre in Schadstoffen (Wien) ohne Verkehrseintrag
missing_traffic_years = (joined
    .filter(F.col("traffic_road_traffic_avg").isNull())
    .select(AIR_YEAR_COL)
    .orderBy(AIR_YEAR_COL))

missing_traffic_years.show(200, truncate=False)

## 13) Spark ML (Regression) – Gerüst

Wir sagen **einen Schadstoff** (Label) aus dem Traffic-Feature voraus.

Du musst nur:
- `LABEL_COL` auf eine Pivot-Spalte setzen (exakt wie im Pivot, z.B. `"NO2"` – je nachdem wie es in deinen Daten heißt)
- optional fehlende Traffic-Jahre imputen (z.B. 0.0) oder Zeilen droppen

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor, LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# ---------------------------
# HIER ANPASSEN
# ---------------------------

# Beispiel: setze hier einen existierenden Schadstoff-Spaltennamen aus dem Pivot:
# LABEL_COL = "NO2"  # <-- nur Beispiel
LABEL_COL = None

# Features: fürs Erste nur der Traffic (du kannst später weitere Features ergänzen)
FEATURE_COLS = ["traffic_road_traffic_avg"]

# Option 1: fehlende Traffic-Werte auf 0 setzen (nur fürs schnelle Testen!)
IMPUTE_MISSING_TRAFFIC_WITH_ZERO = True

In [ ]:
if IMPUTE_MISSING_TRAFFIC_WITH_ZERO:
    model_df = joined.fillna({"traffic_road_traffic_avg": 0.0})
else:
    model_df = joined

print("Spalten im joined:", model_df.columns)

In [ ]:
if LABEL_COL is None:
    print("⚠️ Bitte LABEL_COL setzen. Verfügbare Schadstoff-Spalten (Pivot):")
    # Pivot-Spalten sind alle außer Jahr + traffic feature
    pivot_cols = [c for c in model_df.columns if c not in [AIR_YEAR_COL, "traffic_road_traffic_avg"]]
    print(pivot_cols)
else:
    data = model_df.select([AIR_YEAR_COL, LABEL_COL] + FEATURE_COLS).dropna(subset=[LABEL_COL] + FEATURE_COLS)

    train, test = data.randomSplit([0.8, 0.2], seed=42)

    assembler = VectorAssembler(inputCols=FEATURE_COLS, outputCol="features")
    model = RandomForestRegressor(featuresCol="features", labelCol=LABEL_COL, numTrees=200, maxDepth=8)

    pipeline = Pipeline(stages=[assembler, model])
    fitted = pipeline.fit(train)

    preds = fitted.transform(test)

    rmse = RegressionEvaluator(labelCol=LABEL_COL, predictionCol="prediction", metricName="rmse").evaluate(preds)
    r2 = RegressionEvaluator(labelCol=LABEL_COL, predictionCol="prediction", metricName="r2").evaluate(preds)

    print("✅ RMSE:", rmse)
    print("✅ R2:", r2)

    preds.select(AIR_YEAR_COL, LABEL_COL, "prediction").orderBy(AIR_YEAR_COL).show(200, truncate=False)

## 14) Nächste sinnvolle Erweiterungen
- Mehr Traffic-Features nutzen (z.B. pro District aggregieren, SCWR_CALC etc.)
- Lag-Feature: Verkehr Jahr-1, Jahr-2 (Spark Window)
- Multi-Output: pro Schadstoff ein Modell oder ein Multi-Target Ansatz (außerhalb Spark ML Standard)